In [1]:
from torchPIV.PIVbackend import ToTensor, PIVDataset, moving_window_array
from SIV_library.lib import block_match, moving_reference_array, correlation_to_displacement, get_field_shape, get_x_y

import os
import sys
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

In [78]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

dataset = PIVDataset("../Test Data/plume simulation_PROCESSED", ".jpg",
                     "sequential", transform=ToTensor(dtype=torch.uint8))

cuda


In [252]:
def vector_field_l(fn: str, results: np.ndarray, scale: float, playback_fps = 60.) -> None:
    fig, ax = plt.subplots()

    velocities = results[:, 2:]
    abs_velocities = np.sqrt(velocities[:, 0]**2 + velocities[:, 1]**2)
    min_abs, max_abs = np.min(abs_velocities, axis=0), np.max(abs_velocities, axis=0)
    frames = os.listdir(fn)
    frame = cv2.imread(rf"{fn}/{frames[0]}", cv2.IMREAD_GRAYSCALE)
    image = ax.imshow(frame, cmap='gray')

    x0, y0, vx0, vy0 = results[0]
    vy0 = -vy0
    vectors = ax.quiver(x0, y0, vx0, vy0, max_abs-min_abs, cmap='jet', scale=scale, scale_units='xy')
    def update(index):
        frame = cv2.imread(rf"{fn}/{frames[index]}", cv2.IMREAD_GRAYSCALE)
        image.set_data(frame)

        vx, vy = results[index][2], results[index][3]
        vy = -vy
        abs_vel = np.sqrt(vx**2 + vy**2)
        vectors.set_UVC(vx, vy, abs_vel-min_abs)

        return image, vectors

    _ = animation.FuncAnimation(fig=fig, func=update, frames=results.shape[0], interval=1000/playback_fps)

    plt.show()

In [307]:
def all_frames(window_size, overlap, mode=0):
    results = []
    for i in range(0, 1, 1):
        image_a, image_b = dataset[i]
        
        n_rows, n_cols = get_field_shape(image_a.shape, window_size, overlap)
        window = moving_window_array(image_a, window_size, overlap)
        
        area = moving_reference_array(image_b, window_size, overlap, left=10, right=10, top=10, bottom=10)
        correl = block_match(window, area, mode)
        u, v = correlation_to_displacement(correl, n_rows, n_cols, mode)

        x, y= get_x_y(image_a.shape, window_size, overlap)
        x, y = x.reshape(n_rows, n_cols), y.reshape(n_rows, n_cols)
        
        results.append((x, y, u, v))
    return np.array(results)

In [308]:
res = all_frames(64, 32, mode=1)

100%|██████████| 21/21 [00:17<00:00,  1.23it/s]


3969


In [309]:
fn = "../Test Data/plume simulation_PROCESSED"
vector_field_l(fn, res, 0.1)